# This notebook is used to load data into RedShift

**Note:** Please set kernel to `Python 3 (Data Science)`

After running this notebook, you can save this notebook and push the changes to CodeCommit. This will force the SageMaker Pipeline to run. Instructions to perform the git related operations are in the **instructions.md**

### Variables
Variable name for secret in Secret Manager. RedShift, Athena and Glue information are stored in the secret.

In [1]:
secret_name='bankdm_redshift_login' 

### Install and import libraries

In [2]:
!pip install -q SQLAlchemy==1.3.13
!pip install psycopg2-binary pyathena
!pip install -U pip
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pyathena import connect
from botocore.exceptions import ClientError
import pandas as pd
import json
import boto3
import sagemaker

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils 

### Create client session


In [3]:
# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.client('s3')
redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')

session = boto3.session.Session()
region = session.region_name

### Get credentials & connection information from Secret Manager

In [4]:
try:
    get_secret_value_response = secretsmanager.get_secret_value(
            SecretId=secret_name
        )
    secret_arn=get_secret_value_response['ARN']

except ClientError as e:
    print("Error retrieving secret. Error: " + e.response['Error']['Message'])
    
else:
    # Depending on whether the secret is a string or binary, one of these fields will be populated.
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
    else:
        secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
secret_json = json.loads(secret)
master_user_name = secret_json['username']
master_user_pw = secret_json['password']
redshift_port = secret_json['port']
redshift_cluster_identifier = secret_json['dbClusterIdentifier']
redshift_endpoint_address = secret_json['host']

database_name_redshift = secret_json['database_name_redshift']
database_name_glue = secret_json['database_name_glue']

schema_redshift = secret_json['schema_redshift']
schema_athena = secret_json['schema_athena']

table_name_glue = secret_json['table_name_glue']
table_name_redshift = secret_json['table_name_redshift']

# print(master_user_name)

### Copy data (bank-additional.csv) to S3

In [5]:
s3.upload_file('bank-additional/bank-additional-full.csv', bucket, 'bankdm/bank-additional.csv')


## Athena

### Create Athena database

In [6]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region_name, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name_glue)
print(statement)

CREATE DATABASE IF NOT EXISTS bankdm


In [7]:
import pandas as pd
pd.read_sql(statement, conn)
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,bankdm
1,cagpoc
2,default
3,sagemaker_featurestore


### Load CSV to Athena

The 'default' column name causes an error and I changed the name to 'defaulted' instead.

In [8]:
s3_bankdm_path = "s3://{}/bankdm/".format(bucket)
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         age int,
         job string,
         marital string,
         education string,
         defaulted string,
         housing string,
         loan string,
         contact string,
         month string,
         day_of_week string,
         duration int,
         campaign int,
         pdays int,
         previous int,
         poutcome string,
         emp_var_rate float,
         cons_price_idx float,
         cons_conf_idx float,
         euribor3m float,
         nr_employed int,
         y string
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(database_name_glue, table_name_glue, s3_bankdm_path)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS bankdm.bankdm_glue(
         age int,
         job string,
         marital string,
         education string,
         defaulted string,
         housing string,
         loan string,
         contact string,
         month string,
         day_of_week string,
         duration int,
         campaign int,
         pdays int,
         previous int,
         poutcome string,
         emp_var_rate float,
         cons_price_idx float,
         cons_conf_idx float,
         euribor3m float,
         nr_employed int,
         y string
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '
' 
LOCATION 's3://sagemaker-ap-southeast-1-138604873012/bankdm/'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [9]:
import pandas as pd

pd.read_sql(statement, conn)
statement = "SHOW TABLES in {}".format(database_name_glue)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,bankdm_glue


### Test getting data from Athena

In [10]:
statement = """SELECT * FROM {}.{}
""".format(
    database_name_glue, table_name_glue
)

print(statement)

SELECT * FROM bankdm.bankdm_glue



In [11]:
df = pd.read_sql(statement, conn)
df.head(5)

,age,job,marital,education,defaulted,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,NaN,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None
1,NaN,"Version 2.0 (the ""License""). You",None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None
2,NaN,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None
3,NaN,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None
4,NaN,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None


## RedShift

### Connect to RedShift
#### Before connecting, ensure that the cluster is available

In [12]:
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

available


#### Also ensure the `ApplyStatus` is `in-sync`

In [13]:
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

response['Clusters'][0]['IamRoles']

[{'IamRoleArn': 'arn:aws:iam::138604873012:role/BankDM-RedShift',
  'ApplyStatus': 'in-sync'}]

In [14]:
# print('Redshift endpoint: {}'.format(redshift_endpoint_address))
# print('IAM Role: {}'.format(iam_role))

#### Once the checks are done, connect to RedShift

In [15]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(master_user_name, master_user_pw, redshift_endpoint_address, redshift_port, database_name_redshift))
session = sessionmaker()
session.configure(bind=engine)


### Create RedShift schema

In [16]:
statement = """CREATE SCHEMA IF NOT EXISTS {}""".format(schema_redshift)

s = session()
s.execute(statement)
s.commit()

### Register Athena Database bankdm with Redshift Spectrum to Access the Data Directly in S3 using Glue Data Catalog

With just one command, you can query the S3 data lake from Amazon Redshift without moving any data into our data warehouse. This is the power of Redshift Spectrum. 

Note the `FROM DATA CATALOG` below.  This is pulling the table and schema information from the Glue Data Catalog (ie. Hive Metastore).

In [17]:
statement = """
CREATE EXTERNAL SCHEMA IF NOT EXISTS {} FROM DATA CATALOG 
    DATABASE '{}' 
    IAM_ROLE '{}'
    REGION '{}'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
""".format(schema_athena, database_name_glue, iam_role, region_name)

print(statement)


CREATE EXTERNAL SCHEMA IF NOT EXISTS athena FROM DATA CATALOG 
    DATABASE 'bankdm' 
    IAM_ROLE 'arn:aws:iam::138604873012:role/BankDM-RedShift'
    REGION 'ap-southeast-1'
    CREATE EXTERNAL DATABASE IF NOT EXISTS



In [18]:
s = session()
s.execute(statement)
s.commit()

### Run Sample Query on S3 Data through Redshift Spectrum

In [19]:
statement = """
SELECT *
    FROM {}.{}
""".format(schema_athena, table_name_glue)

print(statement)


SELECT *
    FROM athena.bankdm_glue



In [20]:
df = pd.read_sql_query(statement, engine)
df.head(5)

,age,job,marital,education,defaulted,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,NaN,None,,,,,,,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,None,
1,NaN,"Version 2.0 (the ""License""). You",None,,,,,,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,None,
2,NaN,None,,,,,,,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,None,
3,NaN,None,,,,,,,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,None,
4,NaN,None,,,,,,,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,None,


### Create table in RedShift

In [21]:
statement = """
CREATE TABLE IF NOT EXISTS {}.{}( 
     age integer,
     job text,
     marital text,
     education text,
     defaulted text,
     housing text,
     loan text,
     contact text,
     month text,
     day_of_week text,
     duration integer,
     campaign integer,
     pdays integer,
     previous integer,
     poutcome text,
     emp_var_rate decimal,
     cons_price_idx decimal,
     cons_conf_idx decimal,
     euribor3m decimal,
     nr_employed integer,
     y text
     )
""".format(schema_redshift, table_name_redshift)

print(statement)
s = session()
s.execute(statement)
s.commit()

print("Done.")


CREATE TABLE IF NOT EXISTS dm.data( 
     age integer,
     job text,
     marital text,
     education text,
     defaulted text,
     housing text,
     loan text,
     contact text,
     month text,
     day_of_week text,
     duration integer,
     campaign integer,
     pdays integer,
     previous integer,
     poutcome text,
     emp_var_rate decimal,
     cons_price_idx decimal,
     cons_conf_idx decimal,
     euribor3m decimal,
     nr_employed integer,
     y text
     )

Done.


### Insert data from S3 to RedShift using Athena

In [22]:
statement = """
INSERT INTO {}.{}
    SELECT
        *
    FROM
        {}.{};             

""".format(schema_redshift, table_name_redshift, schema_athena, table_name_glue)
print(statement)
s = session()
s.execute(statement)
s.commit()        
print("Done.")


INSERT INTO dm.data
    SELECT
        *
    FROM
        athena.bankdm_glue;             


Done.


### Test getting data from RedShift

In [23]:
statement = """
SELECT *
    FROM {}.{}
""".format(schema_redshift, table_name_redshift)

print(statement)


SELECT *
    FROM dm.data



In [24]:
df = pd.read_sql_query(statement, engine)
df.head(5)

,age,job,marital,education,defaulted,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56.0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.0,94.0,-36.0,5.0,None,no
1,56.0,services,married,high.school,no,no,yes,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.0,94.0,-36.0,5.0,None,no
2,24.0,technician,single,professional.course,no,yes,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.0,94.0,-36.0,5.0,None,no
3,29.0,blue-collar,single,high.school,no,no,yes,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.0,94.0,-36.0,5.0,None,no
4,35.0,blue-collar,married,basic.6y,no,yes,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.0,94.0,-36.0,5.0,None,no


After running this notebook, you can save this notebook and push the changes to CodeCommit. This will force the SageMaker Pipeline to run. Instructions to perform the git related operations are shown below. These are taken from **instructions.md**

The screenshots show the project as bankdemo2 but yours should be different (e.g. bankdm).

- On the left side, click on the second icon. Scroll your mouse to the right of `Changed`, select the `+` to track all files. Repeat the same for `Untracked`.

![studio](img/studio16.png)

- Your window should look like this where there are no files under `Changed` and `Untracked`. The number of files shown may differ from yours.
- Enter a commit message (commit in the screenshot) and click `Commit`.

![studio](img/studio17.png)

- Enter your name, email and click `OK`.

![studio](img/studio18.png)

- Click on the icon with an up arrow to push the changes. This icon is above the green line in the screenshot.

![studio](img/studio19.png)

- You will get the following message.

![studio](img/studio20.png)

- On the right side of the screen, click on `Pipelines` tab and double click on the Pieplines shown (`BankDemo2-p-sn41sxosn3ef` in my case).

![studio](img/studio21.png)

- The pipeline should automatically run after a short while as you push new codes. Do note that this pipeline will fail as you have not setup RedShift.

![studio](img/studio22.png)